In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
import torch
from transformers import BitsAndBytesConfig
import os
import re
from typing import List, Literal, Optional

from datasets import DatasetDict, concatenate_datasets, load_dataset, load_from_disk
from datasets.builder import DatasetGenerationError


bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the pre-trained LLaMA 3 model and tokenizer
model_name = "defog/llama-3-sqlcoder-8b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)


2024-08-01 12:21:22.747737: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-01 12:21:22.775040: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-01 12:21:23.021419: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-01 12:21:24.215736: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please,

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/cqilab/anaconda3/envs/sam/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/cqilab/anaconda3/envs/sam/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.

In [2]:
EOS_TOKEN = tokenizer.eos_token
eot = "<|eot_id|>"
eot_id = tokenizer.convert_tokens_to_ids(eot)
tokenizer.pad_token = eot
tokenizer.pad_token_id = eot_id
print(EOS_TOKEN)

<|eot_id|>


In [3]:
def format_instruction(sample, response):
    return f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Generate an INSERT/UPDATE/DELETE SQL query ONLY to fulfill this user command: `{sample}`
You will set up the most suitable INSERT/UPDATE/DELETE SQL statements to answer questions related to the data. These SQL statements will later be executed, so they should be error-free, compatible with PostgreSQL syntax, and accurately respond to the questions asked. Do not express an opinion or try to explain. Return only the SQL statement. Remember your sql statement will run on PostgreSQL so syntax should be correct. Only return SQL statement.
Please leave the remaining columns as null ('') if the information about employee is not provided and do not make new assumption (except if default value is provided).
You are a data analyst and interpreter. You will be given information in english or indonesian about a specific dataset, including its tables and their meanings:
employee: List of employee in the app, contains first_name, last_name, email, gender, role, company, working_hours (weekly), and progress.
absensi_harian: Records daily attendance information for employees, capturing absence statuses. Contains employee_id, date, month, year, status (mean ontime or tardiness), overtime_hours, work_hours per day.

Never insert into id column or update it, it is autoincremented.
For Edit/Update, you can use the WHERE clause to specify the row to be updated, it is also possible to update multiple rows at once.
For Edit/Update, user can ask to "add" or "minus" / "tambahkan" or "kurangkan" the value of the column, means you have to increase/decrease specific value using UPDATE statements.
Please leave the remaining columns as null ('') if the information about employee is not provided and do not make new assumption (except if default value is provided).
It is possible to INSERT or UPDATE multiple information or employees at the same time.
If you need to use a specific column, you can refer to it as table_name.column_name.
For anything related best employee performance related, consider to use the most "ontime" first, then "izin", "sakit", "terlambat", "sangat terlambat" as the worst status for employee.progress and work_hours. If there is same status count, then consider the overtime_hours and work_hours must perfectly fit.
For anything related worst employee performance related, consider the most "sangat terlambat" and "terlambat". Also consider the less overtime_hours to overcome the weekly working_hours and work_hours must perfectly fit.
If you cannot fulfill the command with the available database schema or irrelevant command, return 'Mohon maaf, permintaan anda tidak dapat di proses'.

DDL statements:
CREATE TABLE employee (
    id INTEGER NOT NULL, --Unique ID for each employee
    first_name VARCHAR(50), --First name of employee
    last_name VARCHAR(50), --Last name of employee, can be null
    email VARCHAR(50), --Email of employee, can not be duplicated
    gender VARCHAR(50), --Gender of employee, Male or Female
    role VARCHAR(50), --Role of employee in the company like internship, manager-likely, etc. lowercase
    profile_pic text, --Always leave this empty for now
    working_hours INTEGER, --Weekly working hours depend on role as well and base_salary
    company VARCHAR(100) DEFAULT 'Sakura SSS'::VARCHAR, --Company or subgroup of employee
    base_salary INTEGER, --Base salary of employee, also depend on the role
    progress INTEGER, --Current week progress, depends on the working_hours relative to absensi_harian work_hours and overtime
);


CREATE TABLE absensi_harian (
    id INTEGER NOT NULL, --Unique ID
    employee_id INTEGER, --ID of employee
    date date, --date of attendance
    month INTEGER, --month of attendance
    year INTEGER, --year of attendance
    status VARCHAR(20), --status can be 'ontime', 'izin', 'sakit', 'terlambat', 'sangat terlambat', best to worse, affect performance, employee.progress and work_hours
    overtime_hours DECIMAL(10,2), --overtime_hours, the more the better performance and progress, however it is limited to 2 hours, and if too much it is also not good for employee balance
    work_hours DECIMAL(10,2), --work_hours per day, it is affected by absensi_harian.status, it should also fulfill the weekly employee.working_hours if calculated
);


The following SQL query best to fulfill this user command `{sample}`:
```sql{response}<|end_of_text|><|eot_id|>"""

#<|end_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

In [4]:
data = [
    {"sample": "Add a new employee with first name 'Alice' and email 'alice@example.com'.", 
     "response": "\nINSERT INTO employee (first_name, email) VALUES ('Alice', 'alice@example.com');"},
    
    {"sample": "Tambahkan karyawan baru dengan nama depan 'Budi' dan role 'manager'.", 
     "response": "\nINSERT INTO employee (first_name, role) VALUES ('Budi', 'manager');"},
    
    {"sample": "Tambah karyawan Budi", 
     "response": "\nINSERT INTO employee (first_name) VALUES ('Budi');"},
    
    {"sample": "Add employee with first name 'Charlie' and gender 'Male'.", 
     "response": "\nINSERT INTO employee (first_name, gender) VALUES ('Charlie', 'Male');"},
    
    {"sample": "Insert new employee with first_name 'Diana', and company 'Tech Corp'.", 
     "response": "\nINSERT INTO employee (first_name, company) VALUES ('Diana', 'Tech Corp');"},
    
    {"sample": "Add a new employee named 'Eve' with role 'intern'.", 
     "response": "\nINSERT INTO employee (first_name, role) VALUES ('Eve', 'intern');"},
    
    {"sample": "Tambah karyawan baru dengan nama depan 'Fajar' dan gender 'Pria'.", 
     "response": "\nINSERT INTO employee (first_name, gender) VALUES ('Fajar', 'Pria');"},
    
    {"sample": "Add a new employee with first name 'Grace' and base salary 5000.", 
     "response": "\nINSERT INTO employee (first_name, base_salary) VALUES ('Grace', 5000);"},
    
    {"sample": "Insert new employee 'Hana' with email 'hana@company.com'.", 
     "response": "\nINSERT INTO employee (first_name, email) VALUES ('Hana', 'hana@company.com');"},
    
    {"sample": "Tambahkan karyawan baru dengan nama depan 'Iwan' dan progress 10.", 
     "response": "\nINSERT INTO employee (first_name, progress) VALUES ('Iwan', 10);"},
    
    {"sample": "Add employee with first name 'Jack' and company 'Global Inc'.", 
     "response": "\nINSERT INTO employee (first_name, company) VALUES ('Jack', 'Global Inc');"},
    
    {"sample": "Insert new employee with first_name 'Diana', and company 'Tech Corp'.", 
     "response": "\nINSERT INTO employee (first_name, company) VALUES ('Diana', 'Tech Corp');"},
    
    {"sample": "Add a new employee named 'Eve' with role 'intern'.", 
     "response": "\nINSERT INTO employee (first_name, role) VALUES ('Eve', 'intern');"},
    
    {"sample": "Tambah karyawan baru dengan nama depan 'Fajar' dan gender 'Pria'.", 
     "response": "\nINSERT INTO employee (first_name, gender) VALUES ('Fajar', 'Pria');"},
    
    {"sample": "Add a new employee with first name 'Grace' and base salary 5000.", 
     "response": "\nINSERT INTO employee (first_name, base_salary) VALUES ('Grace', 5000);"},
    
    {"sample": "Insert new employee 'Hana' with email 'hana@company.com'.", 
     "response": "\nINSERT INTO employee (first_name, email) VALUES ('Hana', 'hana@company.com');"},
    
    {"sample": "Tambahkan karyawan baru dengan nama depan 'Iwan' dan progress 10.", 
     "response": "\nINSERT INTO employee (first_name, progress) VALUES ('Iwan', 10);"},

      {"sample": "Add employees Alice and Bob with emails 'alice@example.com' and 'bob@example.com'.", 
     "response": "\nINSERT INTO employee (first_name, email) VALUES ('Alice', 'alice@example.com'), ('Bob', 'bob@example.com');"},
    
    {"sample": "Tambahkan karyawan baru Budi dan Cici dengan role 'manager' dan 'intern'.", 
     "response": "\nINSERT INTO employee (first_name, role) VALUES ('Budi', 'manager'), ('Cici', 'intern');"},
    
    {"sample": "Add employees 'David', 'Eli' with gender 'Male', 'Female'.", 
     "response": "\nINSERT INTO employee (first_name, gender) VALUES ('David', 'Male'), ('Eli', 'Female');"},
    
    {"sample": "Insert new employees 'Fiona' and 'George' with company 'Tech Corp'.", 
     "response": "\nINSERT INTO employee (first_name, company) VALUES ('Fiona', 'Tech Corp'), ('George', 'Tech Corp');"},
    
    {"sample": "Add new employees 'Hilda' and 'Ian' with role 'intern'.", 
     "response": "\nINSERT INTO employee (first_name, role) VALUES ('Hilda', 'intern'), ('Ian', 'intern');"},
    
    {"sample": "Tambahkan karyawan baru 'Jack' dan 'Karen' dengan gender 'Pria' dan 'Wanita'.", 
     "response": "\nINSERT INTO employee (first_name, gender) VALUES ('Jack', 'Pria'), ('Karen', 'Wanita');"},
    
    {"sample": "Add employees 'Laura' and 'Mike' with base salary 5000 and 6000.", 
     "response": "\nINSERT INTO employee (first_name, base_salary) VALUES ('Laura', 5000), ('Mike', 6000);"},
    
    {"sample": "Insert new employees 'Nina' and 'Oscar' with emails 'nina@company.com' and 'oscar@company.com'.", 
     "response": "\nINSERT INTO employee (first_name, email) VALUES ('Nina', 'nina@company.com'), ('Oscar', 'oscar@company.com');"},
    
    {"sample": "Tambahkan karyawan baru 'Paula' dan 'Quinn' dengan progress 10 dan 20.", 
     "response": "\nINSERT INTO employee (first_name, progress) VALUES ('Paula', 10), ('Quinn', 20);"},
    
    {"sample": "Add employees 'Robert' and 'Susan' with company 'Global Inc'.", 
     "response": "\nINSERT INTO employee (first_name, company) VALUES ('Robert', 'Global Inc'), ('Susan', 'Global Inc');"},
    
    {"sample": "Delete all employees with the last name 'Smith'.", 
     "response": "\nDELETE FROM employee \nWHERE LOWER(last_name) = 'smith';"},
    
    {"sample": "Hapus semua karyawan dengan email yang mengandung 'example.com'.", 
     "response": "\nDELETE FROM employee \nWHERE LOWER(email) LIKE LOWER('%example.com%');"},
    
    {"sample": "Delete employees with the role 'intern'.", 
     "response": "\nDELETE FROM employee \nWHERE LOWER(role) = 'intern';"},
    
    {"sample": "Hapus karyawan yang bekerja di perusahaan 'Tech Corp'.", 
     "response": "\nDELETE FROM employee \nWHERE LOWER(company) LIKE LOWER('%tech corp%');"},
    
    {"sample": "Delete all female employees.", 
     "response": "\nDELETE FROM employee \nWHERE LOWER(gender) = 'female';"},
    
    {"sample": "Hapus semua karyawan yang memiliki base salary lebih dari 10000.", 
     "response": "\nDELETE FROM employee \nWHERE base_salary > 10000;"},
    
    {"sample": "Delete all employees with the first name 'David'.", 
     "response": "\nDELETE FROM employee \nWHERE LOWER(first_name) = 'david';"},
    
    {"sample": "Hapus karyawan yang memiliki progress kurang dari 5.", 
     "response": "\nDELETE FROM employee \nWHERE progress < 5;"},
    
    {"sample": "Delete employees working in 'HR Department'.", 
     "response": "\nDELETE FROM employee \nWHERE LOWER(role) = 'hr';"},
    
    {"sample": "Hapus semua karyawan yang memiliki working hours lebih dari 40.", 
     "response": "\nDELETE FROM employee \nWHERE working_hours > 40;"},
    
    {"sample": "Tambah progres sebesar 5 untuk semua karyawan di Sakura Internal.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(company) LIKE LOWER('%sakura internal%');"},
    {"sample": "Kurangkan jam kerja sebesar 2 jam untuk semua karyawan yang bekerja di SSS Internal.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(company) LIKE LOWER('%sss internal%');"},
    {"sample": "Kurangkan jam kerja sebesar 3 jam untuk semua karyawan yang bekerja di Sakura Krakatau Steel.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 3 \nWHERE LOWER(company) LIKE LOWER('%sakura krakatau steel%');"},
    {"sample": "Ubah nama depan menjadi 'John' untuk semua karyawan di Sakura Global.", "response": "\nUPDATE employee \nSET first_name = 'John' \nWHERE LOWER(company) LIKE LOWER('%sakura global%');"},
    {"sample": "Tambahkan progress sebesar 10 untuk semua karyawan dengan nama belakang 'Doe'.", "response": "\nUPDATE employee \nSET progress = progress + 10 \nWHERE LOWER(last_name) LIKE LOWER('%doe%');"},
    {"sample": "Kurangkan jam kerja sebesar 4 jam untuk semua karyawan pria di perusahaan SSS.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 4 \nWHERE LOWER(company) LIKE LOWER('%sss%') AND LOWER(gender) = 'male';"},
    {"sample": "Ubah nama depan menjadi 'Alice' untuk semua karyawan dengan email yang mengandung 'example'.", "response": "\nUPDATE employee \nSET first_name = 'Alice' \nWHERE LOWER(email) LIKE LOWER('%example%');"},
    {"sample": "Tambahkan progress sebesar 7 untuk semua karyawan magang.", "response": "\nUPDATE employee \nSET progress = progress + 7 \nWHERE LOWER(role) = 'internship';"},
    {"sample": "Kurangkan jam kerja sebesar 5 jam untuk semua karyawan yang bekerja di bagian manajemen.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 5 \nWHERE LOWER(role) = 'manager';"},
    {"sample": "Ubah nama belakang menjadi 'Smith' untuk semua karyawan dengan gender wanita.", "response": "\nUPDATE employee \nSET last_name = 'Smith' \nWHERE LOWER(gender) = 'female';"},
    {"sample": "Tambahkan progress sebesar 6 untuk semua karyawan di perusahaan yang mengandung 'Tech'.", "response": "\nUPDATE employee \nSET progress = progress + 6 \nWHERE LOWER(company) LIKE LOWER('%tech%');"},
    {"sample": "Kurangkan jam kerja sebesar 1 jam untuk semua karyawan yang bekerja di bagian HR.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE LOWER(role) = 'hr';"},
    {"sample": "Ubah nama depan menjadi 'Michael' untuk semua karyawan dengan nama belakang 'Johnson'.", "response": "\nUPDATE employee \nSET first_name = 'Michael' \nWHERE LOWER(last_name) LIKE LOWER('%johnson%');"},
    {"sample": "Tambahkan progress sebesar 8 untuk semua karyawan di divisi penjualan.", "response": "\nUPDATE employee \nSET progress = progress + 8 \nWHERE LOWER(role) = 'sales';"},
    {"sample": "Kurangkan jam kerja sebesar 3 jam untuk semua karyawan di divisi IT.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 3 \nWHERE LOWER(role) = 'it';"},
    {"sample": "Ubah nama belakang menjadi 'Brown' untuk semua karyawan dengan nama depan 'David'.", "response": "\nUPDATE employee \nSET last_name = 'Brown' \nWHERE LOWER(first_name) = 'david';"},
    {"sample": "Tambahkan progress sebesar 9 untuk semua karyawan di perusahaan yang mengandung 'Global'.", "response": "\nUPDATE employee \nSET progress = progress + 9 \nWHERE LOWER(company) LIKE LOWER('%global%');"},
    {"sample": "Kurangkan jam kerja sebesar 2 jam untuk semua karyawan di perusahaan yang mengandung 'Steel'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(company) LIKE LOWER('%steel%');"},
    {"sample": "Ubah nama depan menjadi 'Emily' untuk semua karyawan dengan email yang mengandung 'corp'.", "response": "\nUPDATE employee \nSET first_name = 'Emily' \nWHERE LOWER(email) LIKE LOWER('%corp%');"},
    {"sample": "Tambahkan progress sebesar 5 untuk semua karyawan dengan nama belakang 'Lee'.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(last_name) LIKE LOWER('%lee%');"},
    {"sample": "Kurangkan jam kerja sebesar 1 jam untuk semua karyawan yang bekerja di bagian finance.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE LOWER(role) = 'finance';"},
    {"sample": "Ubah nama belakang menjadi 'Davis' untuk semua karyawan dengan gender pria.", "response": "\nUPDATE employee \nSET last_name = 'Davis' \nWHERE LOWER(gender) = 'male';"},
    {"sample": "Tambahkan progress sebesar 10 untuk semua karyawan dengan nama depan 'Sara'.", "response": "\nUPDATE employee \nSET progress = progress + 10 \nWHERE LOWER(first_name) = 'sara';"},
    
    {"sample": "Tambah 8 ke progress untuk semua karyawan ontime minggu ini.", "response": "\nUPDATE employee \nSET progress = progress + 8 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'ontime' AND date >= CURRENT_DATE - INTERVAL '7 days');"},
    {"sample": "Kurangi 2 jam kerja untuk semua karyawan di Global.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(company) LIKE LOWER('%global%');"},
    {"sample": "Ubah nama depan jadi 'Linda' untuk semua karyawan wanita dengan nama belakang 'Johnson'.", "response": "\nUPDATE employee \nSET first_name = 'Linda' \nWHERE LOWER(gender) = 'female' AND LOWER(last_name) = 'johnson';"},
    {"sample": "Tambah 10 ke progress untuk semua karyawan di divisi IT.", "response": "\nUPDATE employee \nSET progress = progress + 10 \nWHERE LOWER(role) = 'it';"},
    {"sample": "Kurangi 1 jam kerja untuk semua karyawan di perusahaan yang mengandung 'Corp'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE LOWER(company) LIKE LOWER('%corp%');"},
    {"sample": "Ubah nama belakang jadi 'Taylor' untuk semua karyawan dengan nama depan 'Chris'.", "response": "\nUPDATE employee \nSET last_name = 'Taylor' \nWHERE LOWER(first_name) = 'chris';"},
    {"sample": "Tambah 6 ke progress untuk semua karyawan yang bekerja di perusahaan Steel.", "response": "\nUPDATE employee \nSET progress = progress + 6 \nWHERE LOWER(company) LIKE LOWER('%steel%');"},
    {"sample": "Kurangi 2 jam kerja untuk karyawan wanita di bagian penjualan.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(role) = 'sales' AND LOWER(gender) = 'female';"},
    {"sample": "Ubah nama depan jadi 'Bob' untuk semua karyawan pria dengan nama belakang 'Brown'.", "response": "\nUPDATE employee \nSET first_name = 'Bob' \nWHERE LOWER(gender) = 'male' AND LOWER(last_name) = 'brown';"},
    {"sample": "Tambah 9 ke progress untuk semua karyawan izin bulan ini.", "response": "\nUPDATE employee \nSET progress = progress + 9 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'izin' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Kurangi 1 jam kerja untuk karyawan sakit bulan ini.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'sakit' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Ubah nama belakang jadi 'Clark' untuk semua karyawan pria di divisi IT.", "response": "\nUPDATE employee \nSET last_name = 'Clark' \nWHERE LOWER(role) = 'it' AND LOWER(gender) = 'male';"},
    {"sample": "Tambah 7 ke progress untuk karyawan di perusahaan Global.", "response": "\nUPDATE employee \nSET progress = progress + 7 \nWHERE LOWER(company) LIKE LOWER('%global%');"},
    {"sample": "Kurangi 3 jam kerja untuk semua karyawan pria dengan nama depan 'John'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 3 \nWHERE LOWER(first_name) = 'john' AND LOWER(gender) = 'male';"},
    {"sample": "Ubah nama depan jadi 'Alice' untuk semua karyawan wanita dengan nama belakang 'Lee'.", "response": "\nUPDATE employee \nSET first_name = 'Alice' \nWHERE LOWER(gender) = 'female' AND LOWER(last_name) = 'lee';"},
    {"sample": "Tambah 5 ke progress untuk semua karyawan di divisi HR.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(role) = 'hr';"},
    {"sample": "Kurangi 2 jam kerja untuk karyawan pria di perusahaan yang mengandung 'Tech'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(company) LIKE LOWER('%tech%') AND LOWER(gender) = 'male';"},

     {"sample": "Add 5 progress untuk employee yang berada di Sakura SSS", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(company) LIKE LOWER('%sakura sss%');"},
    {"sample": "Kurang 3 jam kerja untuk karyawan yang berada di SSS Corp", "response": "\nUPDATE employee \nSET working_hours = working_hours - 3 \nWHERE LOWER(company) LIKE LOWER('%sss corp%');"},
    {"sample": "Tambah progress 4 untuk karyawan yang statusnya ontime minggu ini.", "response": "\nUPDATE employee \nSET progress = progress + 4 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'ontime' AND date >= CURRENT_DATE - INTERVAL '7 days');"},
    {"sample": "Ubah nama depan jadi 'Maria' untuk semua karyawan wanita.", "response": "\nUPDATE employee \nSET first_name = 'Maria' \nWHERE LOWER(gender) = 'female';"},
    {"sample": "Tambah 2 progress untuk employee di 'Tech Division'", "response": "\nUPDATE employee \nSET progress = progress + 2 \nWHERE LOWER(company) LIKE LOWER('%tech division%');"},
    {"sample": "Kurangi working hours sebanyak 5 untuk semua employee di Sakura Global", "response": "\nUPDATE employee \nSET working_hours = working_hours - 5 \nWHERE LOWER(company) LIKE LOWER('%sakura global%');"},
    {"sample": "Tambah 6 progress untuk karyawan yang sakit bulan ini.", "response": "\nUPDATE employee \nSET progress = progress + 6 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'sakit' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Ubah last name jadi 'Anderson' untuk employee yang ada di perusahaan 'Internal'.", "response": "\nUPDATE employee \nSET last_name = 'Anderson' \nWHERE LOWER(company) LIKE LOWER('%internal%');"},
    {"sample": "Tambah progress 7 untuk employee dengan email 'company.com'", "response": "\nUPDATE employee \nSET progress = progress + 7 \nWHERE LOWER(email) LIKE LOWER('%company.com%');"},
    {"sample": "Kurangi working hours sebanyak 2 jam untuk semua employee yang statusnya 'izin' minggu ini.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'izin' AND date >= CURRENT_DATE - INTERVAL '7 days');"},
    {"sample": "Add 3 progress untuk semua employee yang ada di divisi 'Finance'", "response": "\nUPDATE employee \nSET progress = progress + 3 \nWHERE LOWER(role) = 'finance';"},
    {"sample": "Ubah first name jadi 'John' untuk employee dengan last name 'Doe'", "response": "\nUPDATE employee \nSET first_name = 'John' \nWHERE LOWER(last_name) = 'doe';"},
    {"sample": "Kurangi working hours 4 jam untuk karyawan di perusahaan 'Global Tech'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 4 \nWHERE LOWER(company) LIKE LOWER('%global tech%');"},
    {"sample": "Tambah 5 progress untuk karyawan di HR Department.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(role) = 'hr';"},
    {"sample": "Ubah nama belakang jadi 'Smith' untuk employee di perusahaan 'Krakatau Steel'.", "response": "\nUPDATE employee \nSET last_name = 'Smith' \nWHERE LOWER(company) LIKE LOWER('%krakatau steel%');"},
    {"sample": "Add 6 progress untuk karyawan yang statusnya 'terlambat' bulan ini.", "response": "\nUPDATE employee \nSET progress = progress + 6 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'terlambat' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Kurangi 1 jam kerja untuk semua employee yang statusnya 'sangat terlambat'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'sangat terlambat');"},
    {"sample": "Tambah 3 progress untuk karyawan di 'Sales Department'", "response": "\nUPDATE employee \nSET progress = progress + 3 \nWHERE LOWER(role) = 'sales';"},
    {"sample": "Ubah nama depan jadi 'Alex' untuk semua employee yang ada di 'Tech Division'", "response": "\nUPDATE employee \nSET first_name = 'Alex' \nWHERE LOWER(company) LIKE LOWER('%tech division%');"},
    {"sample": "Kurangi 2 jam kerja untuk employee yang berada di 'Marketing Department'", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(role) = 'marketing';"},
    {"sample": "Tambah 4 progress untuk karyawan yang statusnya 'ontime' bulan ini.", "response": "\nUPDATE employee \nSET progress = progress + 4 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'ontime' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Ubah nama belakang jadi 'Johnson' untuk semua employee dengan first name 'Michael'.", "response": "\nUPDATE employee \nSET last_name = 'Johnson' \nWHERE LOWER(first_name) = 'michael';"},
    {"sample": "Tambah 5 progress untuk semua karyawan yang bekerja di divisi 'Customer Support'", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(role) = 'customer support';"},
    {"sample": "Kurangi working hours sebanyak 3 jam untuk karyawan di perusahaan 'Finance Corp'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 3 \nWHERE LOWER(company) LIKE LOWER('%finance corp%');"},
    {"sample": "Ubah nama depan jadi 'Chris' untuk karyawan yang ada di divisi 'Operations'.", "response": "\nUPDATE employee \nSET first_name = 'Chris' \nWHERE LOWER(role) = 'operations';"},
    {"sample": "Tambah 6 progress untuk karyawan yang statusnya 'izin' minggu ini.", "response": "\nUPDATE employee \nSET progress = progress + 6 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'izin' AND date >= CURRENT_DATE - INTERVAL '7 days');"},
    {"sample": "Kurangi 2 jam kerja untuk semua employee yang statusnya 'terlambat' bulan ini.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'terlambat' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Ubah nama belakang jadi 'Brown' untuk employee yang ada di perusahaan 'Global Ltd'.", "response": "\nUPDATE employee \nSET last_name = 'Brown' \nWHERE LOWER(company) LIKE LOWER('%global ltd%');"},
    {"sample": "Tambah 7 progress untuk semua karyawan yang bekerja di divisi 'Engineering'", "response": "\nUPDATE employee \nSET progress = progress + 7 \nWHERE LOWER(role) = 'engineering';"},
    {"sample": "Kurangi working hours sebanyak 1 jam untuk semua employee yang statusnya 'sakit'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'sakit');"},
    {"sample": "Ubah nama depan jadi 'Emma' untuk karyawan dengan nama belakang 'Smith'.", "response": "\nUPDATE employee \nSET first_name = 'Emma' \nWHERE LOWER(last_name) = 'smith';"},
    {"sample": "Tambah 5 progress untuk semua karyawan yang bekerja di 'IT Department'", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(role) = 'it';"},
    
    {"sample": "Add 5 progress for all employees in Sakura Internal.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(company) LIKE LOWER('%sakura internal%');"},
    {"sample": "Reduce working hours by 2 hours for all employees working in SSS Internal.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(company) LIKE LOWER('%sss internal%');"},
    {"sample": "Reduce working hours by 3 hours for all employees working in Sakura Krakatau Steel.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 3 \nWHERE LOWER(company) LIKE LOWER('%sakura krakatau steel%');"},
    {"sample": "Change the first name to 'John' for all employees in Sakura Global.", "response": "\nUPDATE employee \nSET first_name = 'John' \nWHERE LOWER(company) LIKE LOWER('%sakura global%');"},
    {"sample": "Add 10 progress for all employees with the last name 'Doe'.", "response": "\nUPDATE employee \nSET progress = progress + 10 \nWHERE LOWER(last_name) LIKE LOWER('%doe%');"},
    {"sample": "Reduce working hours by 4 hours for all male employees in SSS company.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 4 \nWHERE LOWER(company) LIKE LOWER('%sss%') AND LOWER(gender) = 'male';"},
    {"sample": "Change the first name to 'Alice' for all employees with emails containing 'example'.", "response": "\nUPDATE employee \nSET first_name = 'Alice' \nWHERE LOWER(email) LIKE LOWER('%example%');"},
    {"sample": "Add 7 progress for all internship employees.", "response": "\nUPDATE employee \nSET progress = progress + 7 \nWHERE LOWER(role) = 'internship';"},
    {"sample": "Reduce working hours by 5 hours for all employees working in management.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 5 \nWHERE LOWER(role) = 'manager';"},
    {"sample": "Change the last name to 'Smith' for all female employees.", "response": "\nUPDATE employee \nSET last_name = 'Smith' \nWHERE LOWER(gender) = 'female';"},
    {"sample": "Add 6 progress for all employees in companies containing 'Tech'.", "response": "\nUPDATE employee \nSET progress = progress + 6 \nWHERE LOWER(company) LIKE LOWER('%tech%');"},
    {"sample": "Reduce working hours by 1 hour for all employees working in HR.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE LOWER(role) = 'hr';"},
    {"sample": "Change the first name to 'Michael' for all employees with the last name 'Johnson'.", "response": "\nUPDATE employee \nSET first_name = 'Michael' \nWHERE LOWER(last_name) LIKE LOWER('%johnson%');"},
    {"sample": "Add 8 progress for all employees in the sales division.", "response": "\nUPDATE employee \nSET progress = progress + 8 \nWHERE LOWER(role) = 'sales';"},
    {"sample": "Reduce working hours by 3 hours for all employees in the IT division.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 3 \nWHERE LOWER(role) = 'it';"},
    {"sample": "Change the last name to 'Brown' for all employees with the first name 'David'.", "response": "\nUPDATE employee \nSET last_name = 'Brown' \nWHERE LOWER(first_name) = 'david';"},
    {"sample": "Add 9 progress for all employees in companies containing 'Global'.", "response": "\nUPDATE employee \nSET progress = progress + 9 \nWHERE LOWER(company) LIKE LOWER('%global%');"},
    {"sample": "Reduce working hours by 2 hours for all employees in companies containing 'Steel'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(company) LIKE LOWER('%steel%');"},
    {"sample": "Change the first name to 'Emily' for all employees with emails containing 'corp'.", "response": "\nUPDATE employee \nSET first_name = 'Emily' \nWHERE LOWER(email) LIKE LOWER('%corp%');"},
    {"sample": "Add 5 progress for all employees with the last name 'Lee'.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(last_name) LIKE LOWER('%lee%');"},
    {"sample": "Reduce working hours by 1 hour for all employees working in finance.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE LOWER(role) = 'finance';"},
    {"sample": "Change the last name to 'Davis' for all male employees.", "response": "\nUPDATE employee \nSET last_name = 'Davis' \nWHERE LOWER(gender) = 'male';"},
    {"sample": "Add 10 progress for all employees with the first name 'Sara'.", "response": "\nUPDATE employee \nSET progress = progress + 10 \nWHERE LOWER(first_name) = 'sara';"},
    {"sample": "Add 8 progress for all employees on time this week.", "response": "\nUPDATE employee \nSET progress = progress + 8 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'ontime' AND date >= CURRENT_DATE - INTERVAL '7 days');"},
    {"sample": "Reduce working hours by 2 hours for all employees in Global.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(company) LIKE LOWER('%global%');"},
    {"sample": "Change the first name to 'Linda' for all female employees with the last name 'Johnson'.", "response": "\nUPDATE employee \nSET first_name = 'Linda' \nWHERE LOWER(gender) = 'female' AND LOWER(last_name) = 'johnson';"},
    {"sample": "Add 10 progress for all employees in the IT division.", "response": "\nUPDATE employee \nSET progress = progress + 10 \nWHERE LOWER(role) = 'it';"},
    {"sample": "Reduce working hours by 1 hour for all employees in companies containing 'Corp'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE LOWER(company) LIKE LOWER('%corp%');"},
    {"sample": "Change the last name to 'Taylor' for all employees with the first name 'Chris'.", "response": "\nUPDATE employee \nSET last_name = 'Taylor' \nWHERE LOWER(first_name) = 'chris';"},
    {"sample": "Add 6 progress for all employees working in companies containing 'Steel'.", "response": "\nUPDATE employee \nSET progress = progress + 6 \nWHERE LOWER(company) LIKE LOWER('%steel%');"},
    {"sample": "Reduce working hours by 2 hours for female employees in sales.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(role) = 'sales' AND LOWER(gender) = 'female';"},
    {"sample": "Change the first name to 'Bob' for all male employees with the last name 'Brown'.", "response": "\nUPDATE employee \nSET first_name = 'Bob' \nWHERE LOWER(gender) = 'male' AND LOWER(last_name) = 'brown';"},
    {"sample": "Add 9 progress for all employees on leave this month.", "response": "\nUPDATE employee \nSET progress = progress + 9 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'izin' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Reduce working hours by 1 hour for employees who are sick this month.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'sakit' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Change the last name to 'Clark' for all male employees in the IT division.", "response": "\nUPDATE employee \nSET last_name = 'Clark' \nWHERE LOWER(role) = 'it' AND LOWER(gender) = 'male';"},
    {"sample": "Add 7 progress for employees in Global company.", "response": "\nUPDATE employee \nSET progress = progress + 7 \nWHERE LOWER(company) LIKE LOWER('%global%');"},
    {"sample": "Reduce working hours by 3 hours for all male employees with the first name 'John'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 3 \nWHERE LOWER(first_name) = 'john' AND LOWER(gender) = 'male';"},
    {"sample": "Change the first name to 'Alice' for all female employees with the last name 'Lee'.", "response": "\nUPDATE employee \nSET first_name = 'Alice' \nWHERE LOWER(gender) = 'female' AND LOWER(last_name) = 'lee';"},
    {"sample": "Add 5 progress for all employees in HR division.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(role) = 'hr';"},
    {"sample": "Reduce working hours by 2 hours for male employees in companies containing 'Tech'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(company) LIKE LOWER('%tech%') AND LOWER(gender) = 'male';"},
    {"sample": "Add 5 progress for employees in Sakura SSS.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(company) LIKE LOWER('%sakura sss%');"},
    {"sample": "Reduce working hours by 3 hours for employees in SSS Corp.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 3 \nWHERE LOWER(company) LIKE LOWER('%sss corp%');"},
    {"sample": "Add 4 progress for employees with ontime status this week.", "response": "\nUPDATE employee \nSET progress = progress + 4 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'ontime' AND date >= CURRENT_DATE - INTERVAL '7 days');"},
    {"sample": "Change the first name to 'Maria' for all female employees.", "response": "\nUPDATE employee \nSET first_name = 'Maria' \nWHERE LOWER(gender) = 'female';"},
    {"sample": "Add 2 progress for employees in 'Tech Division'.", "response": "\nUPDATE employee \nSET progress = progress + 2 \nWHERE LOWER(company) LIKE LOWER('%tech division%');"},
    {"sample": "Reduce working hours by 5 hours for all employees in Sakura Global.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 5 \nWHERE LOWER(company) LIKE LOWER('%sakura global%');"},
    {"sample": "Add 6 progress for employees who are sick this month.", "response": "\nUPDATE employee \nSET progress = progress + 6 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'sakit' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Change the last name to 'Anderson' for employees in 'Internal'.", "response": "\nUPDATE employee \nSET last_name = 'Anderson' \nWHERE LOWER(company) LIKE LOWER('%internal%');"},
    {"sample": "Add 7 progress for employees with emails containing 'company.com'.", "response": "\nUPDATE employee \nSET progress = progress + 7 \nWHERE LOWER(email) LIKE LOWER('%company.com%');"},
    {"sample": "Reduce working hours by 2 hours for all employees with 'izin' status this week.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'izin' AND date >= CURRENT_DATE - INTERVAL '7 days');"},
    {"sample": "Add 3 progress for all employees in the 'Finance' division.", "response": "\nUPDATE employee \nSET progress = progress + 3 \nWHERE LOWER(role) = 'finance';"},
    {"sample": "Change the first name to 'John' for employees with the last name 'Doe'.", "response": "\nUPDATE employee \nSET first_name = 'John' \nWHERE LOWER(last_name) = 'doe';"},
    {"sample": "Reduce working hours by 4 hours for employees in 'Global Tech'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 4 \nWHERE LOWER(company) LIKE LOWER('%global tech%');"},
    {"sample": "Add 5 progress for employees in the HR Department.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(role) = 'hr';"},
    {"sample": "Change the last name to 'Smith' for employees in 'Krakatau Steel'.", "response": "\nUPDATE employee \nSET last_name = 'Smith' \nWHERE LOWER(company) LIKE LOWER('%krakatau steel%');"},
    {"sample": "Add 6 progress for employees with 'terlambat' status this month.", "response": "\nUPDATE employee \nSET progress = progress + 6 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'terlambat' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Reduce working hours by 1 hour for all employees with 'sangat terlambat' status.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'sangat terlambat');"},
    {"sample": "Add 3 progress for employees in the 'Sales Department'.", "response": "\nUPDATE employee \nSET progress = progress + 3 \nWHERE LOWER(role) = 'sales';"},
    {"sample": "Change the first name to 'Alex' for employees in 'Tech Division'.", "response": "\nUPDATE employee \nSET first_name = 'Alex' \nWHERE LOWER(company) LIKE LOWER('%tech division%');"},
    {"sample": "Reduce working hours by 2 hours for employees in the 'Marketing Department'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE LOWER(role) = 'marketing';"},
    {"sample": "Add 4 progress for employees with 'ontime' status this month.", "response": "\nUPDATE employee \nSET progress = progress + 4 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'ontime' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Change the last name to 'Johnson' for all employees with the first name 'Michael'.", "response": "\nUPDATE employee \nSET last_name = 'Johnson' \nWHERE LOWER(first_name) = 'michael';"},
    {"sample": "Add 5 progress for all employees in the 'Customer Support' division.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(role) = 'customer support';"},
    {"sample": "Reduce working hours by 3 hours for employees in 'Finance Corp'.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 3 \nWHERE LOWER(company) LIKE LOWER('%finance corp%');"},
    {"sample": "Change the first name to 'Chris' for employees in the 'Operations' division.", "response": "\nUPDATE employee \nSET first_name = 'Chris' \nWHERE LOWER(role) = 'operations';"},
    {"sample": "Add 6 progress for employees with 'izin' status this week.", "response": "\nUPDATE employee \nSET progress = progress + 6 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'izin' AND date >= CURRENT_DATE - INTERVAL '7 days');"},
    {"sample": "Reduce working hours by 2 hours for all employees with 'terlambat' status this month.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 2 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'terlambat' AND month = EXTRACT(MONTH FROM CURRENT_DATE));"},
    {"sample": "Change the last name to 'Brown' for employees in 'Global Ltd'.", "response": "\nUPDATE employee \nSET last_name = 'Brown' \nWHERE LOWER(company) LIKE LOWER('%global ltd%');"},
    {"sample": "Add 7 progress for employees in the 'Engineering' division.", "response": "\nUPDATE employee \nSET progress = progress + 7 \nWHERE LOWER(role) = 'engineering';"},
    {"sample": "Reduce working hours by 1 hour for all employees with 'sakit' status.", "response": "\nUPDATE employee \nSET working_hours = working_hours - 1 \nWHERE id IN (SELECT employee_id FROM absensi_harian WHERE status = 'sakit');"},
    {"sample": "Change the first name to 'Emma' for employees with the last name 'Smith'.", "response": "\nUPDATE employee \nSET first_name = 'Emma' \nWHERE LOWER(last_name) = 'smith';"},
    {"sample": "Add 5 progress for all employees in the 'IT Department'.", "response": "\nUPDATE employee \nSET progress = progress + 5 \nWHERE LOWER(role) = 'it';"}
]


In [5]:
model.gradient_checkpointing_enable()

In [6]:
from datasets import Dataset, load_dataset

def formatting_prompts_function(examples):
    texts = []
    for instruction in examples["instruction"]:
        instruction = instruction
        texts.append(instruction)
    return {"text": texts}

formatted_data = [{"instruction": format_instruction(d["sample"], d["response"])} for d in data]
dataset = Dataset.from_dict({"instruction": [f["instruction"] for f in formatted_data]})
dataset = dataset.map(formatting_prompts_function, batched = True)

Map:   0%|          | 0/181 [00:00<?, ? examples/s]

In [7]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to the model
model.enable_input_require_grads()
model = get_peft_model(model, lora_config)

In [8]:
from trl import SFTTrainer
os.environ["WANDB_DISABLED"] = "true"

# Create the Trainer and start fine-tuning
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 4000,
    packing=True,
    args = TrainingArguments(
        evaluation_strategy = "no",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        num_train_epochs = 5,
        warmup_steps = 10,
        learning_rate = 0.0005,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "lora-hris-llama3",
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "cosine",
        seed = 42,
        save_strategy = "steps",
        save_steps = 150,
        save_total_limit = 5,
        report_to="none",
    ),
)

trainer.train()

/home/cqilab/anaconda3/envs/sam/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

[2024-08-01 12:21:39,139] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/cqilab/anaconda3/envs/sam/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


/home/cqilab/anaconda3/envs/sam/lib/python3.11/site-packages/deepspeed/runtime/zero/linear.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @autocast_custom_fwd
/home/cqilab/anaconda3/envs/sam/lib/python3.11/site-packages/deepspeed/runtime/zero/linear.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @autocast_custom_bwd
/home/cqilab/anaconda3/envs/sam/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`use_cache=True` is incompatible with gradient checkpointing. Setting 

Step,Training Loss
1,0.780100
2,0.772700
3,0.777100
4,0.741200
5,0.686000
6,0.667200
7,0.631500
8,0.581900
9,0.530700
10,0.488200


TrainOutput(global_step=55, training_loss=0.16475513770499012, metrics={'train_runtime': 2478.8659, 'train_samples_per_second': 0.093, 'train_steps_per_second': 0.022, 'total_flos': 3.973673189376e+16, 'train_loss': 0.16475513770499012, 'epoch': 4.782608695652174})

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "./lora-hris-llama3/checkpoint-55"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="cuda:0",
    use_cache=True,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/cqilab/anaconda3/envs/sam/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/cqilab/anaconda3/envs/sam/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [10]:
import sqlparse

ddl_statement = """CREATE TABLE employee (
    id INTEGER NOT NULL, --Unique ID for each employee
    first_name VARCHAR(50), --First name of employee
    last_name VARCHAR(50), --Last name of employee, can be null
    email VARCHAR(50), --Email of employee, can not be duplicated
    gender VARCHAR(50), --Gender of employee, Male or Female
    role VARCHAR(50), --Role of employee in the company like internship, manager-likely, etc. lowercase
    profile_pic text, --Always leave this empty for now
    working_hours INTEGER, --Weekly working hours depend on role as well and base_salary
    company VARCHAR(100) DEFAULT 'Sakura SSS'::VARCHAR, --Company or subgroup of employee
    base_salary INTEGER, --Base salary of employee, also depend on the role
    progress INTEGER, --Current week progress, depends on the working_hours relative to absensi_harian work_hours and overtime
);


CREATE TABLE absensi_harian (
    id INTEGER NOT NULL, --Unique ID
    employee_id INTEGER, --ID of employee
    date date, --date of attendance
    month INTEGER, --month of attendance
    year INTEGER, --year of attendance
    status VARCHAR(20), --status can be 'ontime', 'izin', 'sakit', 'terlambat', 'sangat terlambat', best to worse, affect performance, employee.progress and work_hours
    overtime_hours DECIMAL(10,2), --overtime_hours, the more the better performance and progress, however it is limited to 2 hours, and if too much it is also not good for employee balance
    work_hours DECIMAL(10,2), --work_hours per day, it is affected by absensi_harian.status, it should also fulfill the weekly employee.working_hours if calculated
);"""

# @app.route('/generate-sql', methods=['GET'])
def generate_sql(user_question):
    prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Generate an INSERT/UPDATE/DELETE SQL query ONLY to fulfill this user command: `{user_question}`
You will set up the most suitable INSERT/UPDATE/DELETE SQL statements to answer questions related to the data. These SQL statements will later be executed, so they should be error-free, compatible with PostgreSQL syntax, and accurately respond to the questions asked. Do not express an opinion or try to explain. Return only the SQL statement. Remember your sql statement will run on PostgreSQL so syntax should be correct. Only return SQL statement.
Please leave the remaining columns as null ('') if the information about employee is not provided and do not make new assumption (except if default value is provided).
You are a data analyst and interpreter. You will be given information in english or indonesian about a specific dataset, including its tables and their meanings:
employee: List of employee in the app, contains first_name, last_name, email, gender, role, company, working_hours (weekly), and progress.
absensi_harian: Records daily attendance information for employees, capturing absence statuses. Contains employee_id, date, month, year, status (mean ontime or tardiness), overtime_hours, work_hours per day.

Never insert into id column or update it, it is autoincremented.
For Edit/Update, you can use the WHERE clause to specify the row to be updated, it is also possible to update multiple rows at once.
For Edit/Update, user can ask to "add" or "minus" / "tambahkan" or "kurangkan" the value of the column, means you have to increase/decrease specific value using UPDATE statements.
Please leave the remaining columns as null ('') if the information about employee is not provided and do not make new assumption (except if default value is provided).
It is possible to INSERT or UPDATE multiple information or employees at the same time.
If you need to use a specific column, you can refer to it as table_name.column_name.
For anything related best employee performance related, consider to use the most "ontime" first, then "izin", "sakit", "terlambat", "sangat terlambat" as the worst status for employee.progress and work_hours. If there is same status count, then consider the overtime_hours and work_hours must perfectly fit.
For anything related worst employee performance related, consider the most "sangat terlambat" and "terlambat". Also consider the less overtime_hours to overcome the weekly working_hours and work_hours must perfectly fit.
If you cannot fulfill the command with the available database schema or irrelevant command, return 'Mohon maaf, permintaan anda tidak dapat di proses'.

DDL statements:
{ddl_statement}

The following SQL query best to fulfill this user command `{user_question}`:
```sql"""

    input_text = prompt.format(user_question=user_question, ddl_statement=ddl_statement)
    input_ids = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **input_ids,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=1000,
        do_sample=False,
    )
    
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    out = sqlparse.format(outputs[0].split("```sql")[-1], reindent=True)
    return out

In [15]:
input = "Tambah progres sebesar 500 untuk semua karyawan di Sakura Internal dan 200 untuk karyawan di Sakura SSS."
print(generate_sql(input))


UPDATE employee
SET progress = progress + 500
WHERE LOWER(company) ='sakura internal';


: 